## Combining Machine Learning and Optimization 
## With Gurobi and sklearn

## Machine Learning topics 
Touching the elephant here, but ~~not there~~

Supervised Learning
  * Algorithm selection and hyper-parametric optimization
  * KFold assessment vs overfitting
  * Separating training from prediction 

~~Unsupervised Learning~~

~~Time Series Data~~

~~Deep Learning~~

## Optimization topics

  * Exploratory programming to application deployment
  * Coping with the combinatorial explosion
  * Validating optimization with simulation

# All Under the Banner of Python!

## The Soda Promotion Problem

We have the challenge of designing the upcoming promotion campaign for a Soda Company. The intended objective is to bolster sales while at the same time obeying various business constraints.

## The First Challenge

We need to predict impact of different price points on the expected sales for each type of soda.

To do this, we need to train a soda sales predictor from a historical data table.


### Examine historical data 

In [2]:
import pandas
df_hist = pandas.read_excel("soda_sales_historical_data.xlsx")
df_hist[10:]


,Product,Sales,Cost Per Unit,Easter Included,Super Bowl Included,Christmas Included,Other Holiday,4 Wk Avg Temp,4 Wk Avg Humidity,Sales M-1 weeks,Sales M-2 weeks,Sales M-3 weeks,Sales M-4 Weeks,Sales M-5 weeks
10,Alpine Stream,135.7,2.0775,No,No,Yes,No,80.94,69.33,1.9,2.4,2.2,2.0,1.4
11,Bright,3054.8,1.3425,No,No,Yes,No,80.94,69.33,473.4,301.8,188.8,101.4,81.6
12,Crisp Clear,50.3,1.6000,No,No,Yes,No,80.94,69.33,42.2,73.8,69.4,72.8,75.4
13,Popsi Kola,347.9,1.7650,No,No,Yes,No,80.94,69.33,22.9,23.1,22.6,22.1,19.9
14,DC Kola,979.3,1.9250,No,No,Yes,No,80.94,69.33,45.5,56.0,37.7,35.5,21.9
15,Koala Kola,1173.3,2.6725,No,No,Yes,No,80.94,69.33,248.2,344.6,305.3,263.2,290.3
16,AB Root Beer,592.4,3.8625,No,No,Yes,No,80.94,69.33,79.7,74.8,55.8,19.0,18.3
17,Mr. Popper,31.3,2.9750,No,No,Yes,No,80.94,69.33,5.4,7.1,6.4,6.8,7.4
18,11 Down,242.7,1.5300,No,No,Yes,No,81.24,68.78,22.4,10.6,17.0,22.4,13.5
19,Alpine Stream,220.1,1.9100,No,No,Yes,No,81.24,68.78,2.1,1.9,2.4,2.2,2.0


In [ ]:
df_hist.shape

### Convert categorical columns to numeric

In [3]:
from pandas import DataFrame, get_dummies
categorical_columns = ['Product','Easter Included','Super Bowl Included', 
                       'Christmas Included', 'Other Holiday']
df_hist = get_dummies(df_hist, prefix={k:"dmy_%s"%k for k in categorical_columns},
                      columns = list(categorical_columns))
df_hist[:5]


,Sales,Cost Per Unit,4 Wk Avg Temp,4 Wk Avg Humidity,Sales M-1 weeks,Sales M-2 weeks,Sales M-3 weeks,Sales M-4 Weeks,Sales M-5 weeks,dmy_Product_11 Down,...,dmy_Product_Koala Kola,dmy_Product_Mr. Popper,dmy_Product_Popsi Kola,dmy_Easter Included_No,dmy_Easter Included_Yes,dmy_Super Bowl Included_No,dmy_Super Bowl Included_Yes,dmy_Christmas Included_No,dmy_Christmas Included_Yes,dmy_Other Holiday_No
0,51.9,1.6625,80.69,69.19,17.0,22.4,13.5,14.5,28.0,1,...,0,0,0,1,0,1,0,0,1,1
1,55.8,2.2725,80.69,69.19,2.4,2.2,2.0,1.4,0.5,0,...,0,0,0,1,0,1,0,0,1,1
2,3385.6,1.3475,80.69,69.19,301.8,188.8,101.4,81.6,213.8,0,...,0,0,0,1,0,1,0,0,1,1
3,63.5,1.6600,80.69,69.19,73.8,69.4,72.8,75.4,57.4,0,...,0,0,0,1,0,1,0,0,1,1
4,181.1,1.8725,80.69,69.19,23.1,22.6,22.1,19.9,23.2,0,...,0,0,1,1,0,1,0,0,1,1


### Picking the right predictor algorithm is of upmost importance

Hence we examine our choices here in great deal.

In [4]:
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import BaggingRegressor
# from sklearn import model_selection
# experiments = {"Algorithm":["Ordinary Least Squares", "Regression Tree", 
#                             "Big Random Forest", "Random Forest", 
#                             "Bagging"], 
#                "Objects" : [lambda : LinearRegression(), 
#                             lambda : DecisionTreeRegressor(), 
#                             lambda : RandomForestRegressor(n_estimators=100), 
#                             lambda : RandomForestRegressor(), 
#                             lambda : BaggingRegressor()], 
#                "Predictions":[[] for _ in range(5)]}
# actuals = []

from torch import *
import torch.nn as nn

len(df_hist)

596

### Resist the temptation to overfit!
Instead, split the samples into train, test subsections.

In [5]:
# from sklearn.model_selection import train_test_split
# [_.shape for _ in  train_test_split(df_hist.drop("Sales", axis=1), 
#                                     df_hist["Sales"], test_size=0.25)]
import torch.utils.data
trainSize = int(0.75 * len(df_hist))
testSize = len(df_hist) - trainSize
target = df_hist["Sales"]
dropped = df_hist.drop("Sales", axis=1)
# trainDataSet, testDataSet = torch.utils.data.random_split(dropped, [trainSize, testSize])
# print(type(df_hist))
# targetCols = ['Sales']
# trainingCols = []
# for columnHeaders in df_hist.columns:
#     if columnHeaders != 'Sales':
#         trainingCols.append(columnHeaders)
# print(targetCols)
# print(trainingCols)
print(type(target))
targetTensor = torch.tensor(target.values).float()
targetTensor = targetTensor.resize_((596,1))
print(targetTensor.shape)
trainingTensor = torch.tensor(dropped.values).float()
print(trainingTensor.shape)

<class 'pandas.core.series.Series'>
torch.Size([596, 1])
torch.Size([596, 24])


#### By repeatedly splitting, training, and testing, you can create a realistic simulation of prediction accuracy.

In [6]:
# for _ in range (4):
#     train_X, test_X, train_y, test_y = (
#         train_test_split(df_hist.drop("Sales", axis=1), 
#                          df_hist["Sales"], test_size=0.25))
#     for i, obj_factory in enumerate(experiments["Objects"]):
#         obj = obj_factory()
#         obj.fit(y=train_y,X=train_X)
#         experiments["Predictions"][i] += list(obj.predict(test_X))
#     actuals += list(test_y)
# actuals = pandas.Series(actuals)
# experiments["Predictions"] = list(map(pandas.Series, experiments["Predictions"]))
# # trainDataSet
from torch.utils.data import *
# Define dataset
train_ds = TensorDataset(trainingTensor, targetTensor)
train_ds[0:3]

(tensor([[  1.6625,  80.6900,  69.1900,  17.0000,  22.4000,  13.5000,  14.5000,
           28.0000,   1.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   1.0000,   0.0000,
            0.0000,   1.0000,   1.0000],
         [  2.2725,  80.6900,  69.1900,   2.4000,   2.2000,   2.0000,   1.4000,
            0.5000,   0.0000,   0.0000,   1.0000,   0.0000,   0.0000,   0.0000,
            0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   1.0000,   0.0000,
            0.0000,   1.0000,   1.0000],
         [  1.3475,  80.6900,  69.1900, 301.8000, 188.8000, 101.4000,  81.6000,
          213.8000,   0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   0.0000,
            0.0000,   0.0000,   0.0000,   1.0000,   0.0000,   1.0000,   0.0000,
            0.0000,   1.0000,   1.0000]]), tensor([[  51.9000],
         [  55.8000],
         [3385.6001]]))

In [7]:
# 
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[ 2.7150, 80.2000, 68.0100, 67.7000, 68.0000, 81.7000, 64.7000, 79.1000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
           0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
         [ 4.1025, 80.2000, 68.0100, 11.6000, 15.2000, 18.6000,  8.4000, 10.1000,
           0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
         [ 2.8925, 81.5900, 67.5900,  7.4000,  7.9000, 18.0000, 16.6000,  7.4000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
           0.0000,  1.0000,  0.0000,  0.0000,  1.0000,  1.0000,  0.0000,  1.0000],
         [ 1.6000, 76.0300, 68.4300,  4.3000, 34.1000,  6.0000,  7.3000, 15.6000,
           1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000,  1.0000],
         [ 1

In [37]:
# Define model

model = nn.Linear(24, 1)  # 24 inputs, 1 output
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1083,  0.1404, -0.0648,  0.1107, -0.1041,  0.1893,  0.1841, -0.2018,
          0.1767, -0.1248,  0.0291, -0.1982, -0.0098, -0.1594,  0.1781,  0.0496,
         -0.1231, -0.0706, -0.1572,  0.1575, -0.1679,  0.0292, -0.0371,  0.0064]],
       requires_grad=True)
Parameter containing:
tensor([0.1165], requires_grad=True)


In [21]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)
# using stochastic gradient descent (SGD) as the optimizer

In [22]:
# Import nn.functional
import torch.nn.functional as F

In [23]:
# Define loss function
loss_fn = F.mse_loss
# mean squared error as the loss function

In [38]:
loss = loss_fn(model(trainingTensor), targetTensor)
print(trainingTensor.float().shape)
print(targetTensor.float().shape)
print(loss)
# print(targetTensor)


torch.Size([596, 24])
torch.Size([596, 1])
tensor(756662.9375, grad_fn=<MseLossBackward>)


In [40]:
# Define a utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
    print('Training loss: ', loss_fn(model(trainingTensor), targetTensor))

In [43]:
# Train the model for 100 epochs
fit(100, model, loss_fn, opt)

Training loss:  tensor(756662.9375, grad_fn=<MseLossBackward>)


In [58]:
# Generate predictions
preds = model(trainingTensor)
preds[1][0].data

tensor(7.8624)

In [59]:
# Compare with targets
targetTensor[1][0].data

tensor(55.8000)

In [60]:
df_superbowl_original = pandas.read_excel("super_bowl_promotion_data.xlsx")
df_superbowl = get_dummies(df_superbowl_original, 
                           prefix={k:"dmy_%s"%k for k in categorical_columns},
                           columns = list(categorical_columns))
assert "Sales" not in df_superbowl.columns 
assert {"Sales"}.union(df_superbowl.columns).issubset(set(df_hist.columns))
len(df_superbowl)

36

Note that the current data table might have less categorical range than the historical data. 

In [61]:
for fld in set(df_hist.columns).difference(df_superbowl.columns, {"Sales"}):
    assert fld.startswith("dmy_")
    df_superbowl[fld] = 0

**Take care!!** `sklearn` [has no concept of columns](https://github.com/scikit-learn/scikit-learn/issues/7242). We make sure that the `df_superbowl` columns are ordered consistently with the `df_hist` independent column sub-matrix.

In [72]:
df_superbowl = df_superbowl[list(df_hist.drop("Sales", axis=1).columns)]
# print(df_superbowl)
testingTensor = torch.tensor(df_superbowl.values).float()
testingTensor

tensor([[1.3425e+00, 8.0790e+01, 7.1120e+01, 2.1525e+01, 2.2075e+01, 3.0125e+01,
         3.9400e+01, 3.8775e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00],
        [1.7125e+00, 8.0790e+01, 7.1120e+01, 4.8775e+01, 3.8025e+01, 3.1025e+01,
         3.1300e+01, 2.7550e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00],
        [1.9975e+00, 8.0790e+01, 7.1120e+01, 1.5525e+01, 6.4750e+00, 5.1750e+00,
         6.5250e+00, 5.3000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
         0.0000e+00, 0.0000e+00, 1.0000e+00, 1.0000e+00, 0.0000e+00, 1.0000e+00],
        [2.5650e+00, 8.07

In [74]:
predicted = model(testingTensor)
predicted

tensor([[ 12.0020],
        [ 14.2297],
        [  9.0473],
        [ 51.2663],
        [165.0101],
        [  8.4197],
        [ 51.2630],
        [  9.0638],
        [ 50.3742],
        [165.0093],
        [ 38.6505],
        [  8.4169],
        [  9.0625],
        [  9.4434],
        [ 14.2337],
        [ 12.0112],
        [ 14.2313],
        [  9.4583],
        [ 51.2574],
        [165.0082],
        [  9.4491],
        [ 51.2658],
        [ 38.6548],
        [ 11.9987],
        [ 50.3734],
        [ 12.0158],
        [  8.4221],
        [  9.4486],
        [ 14.2253],
        [ 50.3696],
        [ 38.6627],
        [  8.4107],
        [ 50.3723],
        [  9.0722],
        [  9.4483],
        [ 38.6594]], grad_fn=<AddmmBackward>)

# Prediction in hand, we commence optimization!

### LaTeX summary of family of equations

$$
\begin{array}{ll}
\max & sales\\
s.t. & X_{b,p}\in\{0,1\}\quad\forall (b,p)\in Prod\\
     & \sum\left(X_{b,p}:{(b,p)\in Prod} \right)=1\quad\forall b\in Soda\\
     & \sum\left(X_{b,p}:{(b,p)\in Prod, p\neq p_o, T(b)=t} \right) \leq max_t\quad\forall t\\
     & sales = \sum\left(f_{b,p} X_{b,p}:{(b,p)\in Prod}\right)\\
     & revenue = \sum\left(f_{b,p}  p X_{b,p}:{(b,p)\in Prod} \right)\\
     & investment = \sum\left(\left(f_{b,p} - f_{b,p_o}\right)_+ p_o X_{b,p}:{(b,p)\in Prod} \right)\\
     & investment <= max_{investment}
\end{array}
$$

### Putting the optimization input set together

In [75]:
forecast_sales = df_superbowl_original[["Product", "Cost Per Unit"]].copy()
forecast_sales["Sales"] = predicted
forecast_sales.set_index(['Product','Cost Per Unit'], inplace=True)

RuntimeError: Can't call numpy() on Variable that requires grad. Use var.detach().numpy() instead.

In [ ]:
soda_family = {'11 Down': 'Clear', 'AB Root Beer': 'Dark', 
               'Alpine Stream': 'Clear', 'Bright': 'Clear', 
               'Crisp Clear': 'Clear', 'DC Kola': 'Dark',
               'Koala Kola': 'Dark', 'Mr. Popper': 'Dark', 
               'Popsi Kola': 'Dark'}
family  = set(soda_family[j] for j in soda_family)
soda    = set(j for j in soda_family)
max_prom = {f:2 for f in family}
max_investment = 750

In [ ]:
product_prices = set(forecast_sales.index.values)
normal_price = {b:0 for b in soda}
for b,p in product_prices:
    normal_price[b] = max(normal_price[b],p)

Note that not all estimated discounts yield a boost in sales.

In [ ]:
meaningful_discounts = 0
for b,p in product_prices:
    if forecast_sales.Sales[b,p] > forecast_sales.Sales[b,normal_price[b]]:
        meaningful_discounts += 1
meaningful_discounts, len(forecast_sales) - len(soda)

### Building a MIP model

$$
\begin{array}{l}
X_{b,p}\in\{0,1\}\quad\forall (b,p)\in Prod\\
0 \leq sales\\
0 \leq revenue\\
0 \leq investment \leq max_{investment}
\end{array}
$$

In [ ]:
import gurobipy as gu
model = gu.Model()
select_price = model.addVars(product_prices,vtype=gu.GRB.BINARY,name='X')
sales        = model.addVar(name='sales')
revenue      = model.addVar(name='revenue')
investment   = model.addVar(ub=max_investment, name='investment')
gusum = gu.quicksum

$$ 
sales = \sum\left(f_{b,p} X_{b,p}:{(b,p)\in Prod}\right)\\
revenue = \sum\left(f_{b,p} p X_{b,p}:{(b,p)\in Prod} \right)\\
investment = \sum\left(\left(f_{b,p} - f_{b,p_o}\right)_+ p_o X_{b,p}:{(b,p)\in Prod} \right)
$$

In [ ]:
model.addConstr(sales == select_price.prod(forecast_sales.Sales), name='sales')
model.addConstr(revenue == gusum(forecast_sales.Sales[b,p] * p * 
                                 select_price[b,p] for b,p in product_prices), 
                name='revenue')
model.addConstr(investment == 
                gusum(max(0,forecast_sales.Sales[b,p] - 
                            forecast_sales.Sales[b,normal_price[b]]) *
                        normal_price[b] * select_price[b,p] 
                        for b,p in product_prices),
                name='investment')
model.update()

$$
\sum\left(X_{b,p}:{(b,p)\in Prod} \right)=1\quad\forall b\in Soda\\
\sum\left(X_{b,p}:{(b,p)\in Prod, p\neq p_o, T(b)=t} \right) \leq max_t
$$

In [ ]:
model.addConstrs((select_price.sum(b,'*') == 1 for b in soda), name='OnePrice')
model.addConstrs((gusum(select_price[b,p] for b,p in product_prices if 
                        soda_family[b] == f and p != normal_price[b] ) 
                  <= max_prom[f] for f in family),
                 name='MaxProm')
model.update()

### Optimize and results

In [ ]:
model.setObjective(sales, sense=gu.GRB.MAXIMIZE)
model.optimize()

In [ ]:
model.status == gu.GRB.OPTIMAL

### Only the paranoid survive
Carefully sanity check the solution.

In [ ]:
sales.X, revenue.X, investment.X

In [ ]:
price_selections = {"Product":[], "Price":[], "Is Discount":[], "Family":[]}
for b, p in product_prices:
    if abs(select_price[b,p].X -1) < 0.0001: # i.e. almost one
        price_selections["Product"].append(b)
        price_selections["Price"].append(p)
        price_selections["Is Discount"].append(p < normal_price[b])
        price_selections["Family"].append(soda_family[b])
(DataFrame(price_selections).set_index("Product")
 [["Price", "Is Discount", "Family"]].sort_values("Family"))

#### Create a range of predictions to simulate the behavior of our solution under a range of conditions.

In [ ]:
simulated_KPI = {'Sales':[],'Revenue':[],'Investment':[]}
Z = select_price
num_infeas = 0
for i in range(100):
    np.random.seed(i)
    fitted = RandomForestRegressor(n_estimators=100,
                                   n_jobs=4).fit(y=df_hist["Sales"],
                                                 X=df_hist.drop("Sales", axis=1))
    forecast = df_superbowl_original[['Product', 'Cost Per Unit']].copy()
    forecast["Sales"] = fitted.predict(df_superbowl)
    forecast = forecast.set_index(['Product','Cost Per Unit'])
    sales, revenue, investment = 0, 0, 0
    for b,p in product_prices:
        sales   += forecast.Sales[b,p] * Z[b,p].X
        revenue += forecast.Sales[b,p] * p * Z[b,p].X
        investment += (max(0,forecast.Sales[b,p] - 
                             forecast.Sales[b,normal_price[b]]) * 
                       normal_price[b] * Z[b,p].X)
    if investment > max_investment:
        num_infeas += 1
    simulated_KPI['Sales'].append(sales)
    simulated_KPI['Revenue'].append(revenue)
    simulated_KPI['Investment'].append(investment)

In [ ]:
data = {'Sales','Revenue','Investment'}
color=iter(cm.rainbow(np.linspace(0,1,3)))
for t in data:
    plt.figure(figsize=(7,4),dpi=300)
    plt.hist(simulated_KPI[t],50,normed=1,color=next(color), alpha=0.75)
    plt.ylabel('Probability')
    plt.xlabel(t)
    plt.grid(True)
    plt.show()

In [ ]:
num_infeas

## Thank you for joining us
 * If you haven't already done so, please register at http://www.gurobi.com/ and then visit http://www.gurobi.com/downloads/get-anaconda to try Gurobi and Python for yourself.
 * Explore ticdat at https://pypi.python.org/pypi/ticdat/ and see the Opalytics Cloud Platform in action at http://bit.ly/2sjEuZt. 
 * For questions about Gurobi Pricing contact sales@gurobi.com or sales@gurobi.de.
 * A recording of the webinar, including the slides, will be available in roughly one week.

# Some extra references for Stochastic, Robust, and Risk optimization

After the webinar we received a lot of requests for further material on this topic. The following list is an (incomplete) list of sources for these topics, but a good starting point for it.

## Books

<ul>
    <li>Stochastic Programming, Peter Kall, Stein W. Walace, 1994</li>
    <li>Optimization Methods in Finance, Gerard Cornuejols and Reha T&uuml;t&uuml;nc&uuml;, 2006</li>
    <li>Lectures on Stochastic Programming: Modeling and Theory, Alexander Shapiro, Darinka Dentcheva, Andrzej Ruszczy&nacute;ski, 2009</li>
    <li>Robust Optimization, Aharon Ben-Tal, Laurent El Ghaoui, Arkadi Nemirovski, 2009</li>
    <li>Introduction to Stochastic Programming, John Birge, Fran&ccedil;ois Louveaux, 2011</li>
    <li>Modeling with Stochastic Programming, Alan J. King, Stein W. Walace, 2012</li>
</ul>

## Some papers

<ul>
    <li>Optimization of Conditional Value-at-Risk, R. Tyrrell Rockafellar, Stanislav Uryasev, 2000</li>
    <li>Supplier-Retailer Flexible Commitments Contracts: A Robust Optimization Approach, Aharon Ben Tal , Boaz Golany, Arcadi Nemirovskiy, Jean-Philippe Vial, 2003</li>
    <li>Tractable Approximations to Robust Conic Optimization Problems,Dimitris Bertsimas, Melvyn Sim, 2006</li>
    <li>Modeling and optimization of risk, Pavlo Krokhmal, Michael Zabarankin,Stan Uryasev, 2011</li>
</ul>

## Other resources

The stochastic optimization society in their <a href="www.stoprog.org">web site</a> has several tutorials and further links.

### Special Thanks

I would like to specially thank (in alphabetic order) to
<a href="http://www2.isye.gatech.edu/people/faculty/Shabbir_Ahmed/">Shabbir Ahmed</a>, Georgia Tech, 
<a href="http://www.uai.cl/academicos/cuerpo-academico/tito-homem-de-mello">Tito Homem-de-mello</a>, Universidad Adolfo Iba&ntilde;ez, 
<a href="https://directory.engr.wisc.edu/ie/faculty/luedtke_james">Jim Luedtke</a>, University of Wisconsin-Madison,
<a href="http://www.mccormick.northwestern.edu/research-faculty/directory/profiles/morton-david.html">David Morton</a>,  Northwestern University, and
<a href="http://bernardokp.uai.cl/">Bernardo Pagnoncelli</a>, Universidad Adolfo Iba&ntilde;ez, for suggesting books, links and papers. Any omission or error is my fault.

Daniel Espinoza,<br>
Senior Developer,<br>
Gurobi Optimization Inc.
